# District by Section

In [ ]:
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql
DROP TABLE IF EXISTS report_districts;
CREATE TEMP TABLE report_districts AS 
SELECT 
  r.district as district,
  r.assigned_section as section,
  COUNT(*) count
FROM cts_forms_report r
WHERE district notnull
GROUP BY district, section
HAVING COUNT(*) > 1
ORDER BY count desc;

SELECT * FROM report_districts;

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('_mpl-gallery')

plt.rcParams['figure.figsize'] = [4, 24]

def make_district_chart(*, section='(any)', scale=1):
    all_reports = %sql SELECT * \
      FROM report_districts \
      WHERE \
        '{section}' = '(any)' \
        OR ('{section}' != '(any)' AND section='{section}');
    data = all_reports.DataFrame()
    count = list(data['count'])
    districts = list(data['district'])
    y = list(districts)
    x = list(count)
    fig, ax = plt.subplots()
    ax.barh(y, x)
    ax.set_xlabel('Number of Reports')
    ax.set_ylabel('District')
    ax.set_title(f'Reports by District for: {section}')
    return plt


In [ ]:
import ipywidgets

sections = %sql SELECT section FROM report_districts GROUP BY 1;
sections = sections.DataFrame().section

section_dropdown = ipywidgets.Dropdown(
    options=['(any)', *sections],
    value='(any)',
    description='Assigned Section:',
    disabled=False,
)

def observe_dropdown(*, section):
    plt.rcParams['figure.figsize'] = [8, 10]
    district_chart = make_district_chart(section=section)
    district_chart.show()

ipywidgets.interact(observe_dropdown, section=section_dropdown)
pass
